In [1]:
from huggingface_hub import notebook_login
from dotenv import load_dotenv
import os

import torch 
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments

from peft import prepare_model_for_kbit_training
from peft import LoraConfig
from peft import get_peft_model

from trl import SFTTrainer

In [2]:
# Load environment variables from .env file
load_dotenv()
# 
# Retrieve the OPENAI_API_KEY
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

HUGGINGFACE_WRITE_TOKEN = os.getenv("HUGGINGFACE_WRITE_TOKEN")
# print(HUGGINGFACE_WRITE_TOKEN)

WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")
WEAVIATE_CLUSTER = os.getenv("WEAVIATE_CLUSTER")



In [4]:
data = load_dataset("samsum", split="train")
data

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [5]:
df = data.to_pandas()
df.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [6]:
df['text'] = df[['dialogue', 'summary']].apply(lambda x: "###Human: Summarize this following dialogue: " + x["dialogue"] + "\n###Assistant: " +x["summary"], axis=1)

In [7]:
df['text'][0]

"###Human: Summarize this following dialogue: Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)\n###Assistant: Amanda baked cookies and will bring Jerry some tomorrow."

In [8]:
data = Dataset.from_pandas(df)
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text'],
    num_rows: 14732
})

In [9]:
import torch
print(torch.cuda.is_available())  # Should return True


True


# llm Model and Tokenizer

In [10]:
# # from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# # from optimum.gptq import GPTQConfig



# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

# # Configure quantization
# quantization_config = GPTQConfig(
#     bits=4, 
#     disable_exllama=False,  # Disable Exllama as per the error suggestion
#     tokenizer=tokenizer
# )

# # Load model with device map
# model = AutoModelForCausalLM.from_pretrained(
#     "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
#     quantization_config=quantization_config,
#     device_map="auto"
# )

In [11]:
# aa

In [12]:
# https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GPTQ
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

In [13]:
tokenizer.eos_token

'</s>'

In [14]:
tokenizer.pad_token = tokenizer.eos_token

In [15]:
# https://huggingface.co/docs/transformers/en/main_classes/quantization
quantization_confiq_loading = GPTQConfig(bits=4, use_exllama= True, tokenizer=tokenizer)

In [16]:

model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                                             quantization_config=quantization_confiq_loading,
                                             device_map="auto",
                                             use_auth_token=HUGGINGFACE_WRITE_TOKEN)

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\models\auto\auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\quantizers\auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)
CUDA extension not installed.
CUDA extension not installed.
c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\modeling_utils.py:4492: FutureWarning: `_is_quantized_training_enabled` is g

In [17]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


In [18]:
model.config.use_cache = False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()

## Quantization with peft LoRa

In [19]:
model = prepare_model_for_kbit_training(model)
# model

In [20]:
peft_config = LoraConfig(r=16, 
                         lora_alpha=18, 
                         lora_dropout=0.05, 
                         bias="none", 
                         task_type="CASUAL_LM", 
                         target_modules= ["q_proj", "v_proj"]
                         )

In [21]:
model = get_peft_model(model, peft_config)
model

PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict(

## Training

In [22]:
training_arguments = TrainingArguments(output_dir="misteral-finetuned-samsum",
                                       per_device_train_batch_size=8,
                                       gradient_accumulation_steps=1,
                                       optim="paged_adamw_32bit",
                                       learning_rate=2e-4,
                                       warmup_ratio=0.1,
                                       lr_scheduler_type="cosine",
                                       save_strategy="epoch",
                                       weight_decay=0.01,
                                       logging_steps=100,
                                       num_train_epochs=1,
                                       max_steps=259,
                                       fp16=True,
                                       push_to_hub=True
                                       )

In [23]:
SFTTrainer(model=model,
           train_dataset=data,
           peft_config=peft_config,
           dataset_text_field="text",
           args=training_arguments,
           tokenizer=tokenizer,
           packing=False,
           max_seq_length=512
           )

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\trl\trainer\sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
